<a href="https://colab.research.google.com/github/rewpak/AI-works/blob/main/Gen_Algorithms_Travelling_Salesman_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3. Genetic Algorithms
# Task 3.1 The Travelling Salesman Problem
## Problem Descriptions
In this task, we are presented with the Traveling Salesman Problem (TSP). The objective of the TSP is to determine the shortest possible route that a salesman can take to visit 17 cities and return to the starting city, covering each city exactly once.

Using GA, firstly we need to determine the encoding scheme, evaluation of fitness, and the genetic operators:

1. Encoding scheme: Create every possible order of the 17 numbers that stand for the cities.

symbolic encoding:

(1) 1 4 3 2 7 5 6 11 8 10 9 12 15 13 14 17 16

(2) 3 4 7 8 17 ... and etc
2. Fitness function: the sum of distances of all neighboring cities in a string, including the distance between starting and end cities.

3. Genetic operators:
    * Selection: tournament selection, a random subset of individuals is selected from the population
    * Crossover: choose a crossover point randomly, the sub-strings before this point are copied from one parent, and the remaining part is filled in following the order in the second parent.
    * Mutation: two random positions are chosen, and the values at those positions are swapped.

## Implementation and Results

In [ ]:
TourSize = 17
OptTour = [0, 15, 11, 8, 4, 1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3],
OptDistance = 2085
distance_map = \
    [[0, 633, 257, 91, 412, 150, 80, 134, 259, 505, 353, 324, 70, 211, 268, 246, 121],
    [633, 0, 390, 661, 227, 488, 572, 530, 555, 289, 282, 638, 567, 466, 420, 745, 518],
    [257, 390, 0, 228, 169, 112, 196, 154, 372, 262, 110, 437, 191, 74, 53, 472, 142],
    [91, 661, 228, 0, 383, 120, 77, 105, 175, 476, 324, 240, 27, 182, 239, 237, 84],
    [412, 227, 169, 383, 0, 267, 351, 309, 338, 196, 61, 421, 346, 243, 199, 528, 297],
    [150, 488, 112, 120, 267, 0, 63, 34, 264, 360, 208, 329, 83, 105, 123, 364, 35],
    [80, 572, 196, 77, 351, 63, 0, 29, 232, 444, 292, 297, 47, 150, 207, 332, 29],
    [134, 530, 154, 105, 309, 34, 29, 0, 249, 402, 250, 314, 68, 108, 165, 349, 36],
    [259, 555, 372, 175, 338, 264, 232, 249, 0, 495, 352, 95, 189, 326, 383, 202, 236],
    [505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 154, 578, 439, 336, 240, 685, 390],
    [353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 435, 287, 184, 140, 542, 238],
    [324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 254, 391, 448, 157, 301],
    [70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 145, 202, 289, 55],
    [211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 57, 426, 96],
    [268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 483, 153],
    [246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 336],
    [121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0]]

In [ ]:
!pip install deap
import array
import random
import numpy as np
from deap import algorithms, base, creator, tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
# fitness function as the sum of all the distances between each consecutive cities in individual
def evalTSP(individual):
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

# create creator.FitnessMin and creator.Individual to be used in the toolbox
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

# create the toolbox from the base
toolbox = base.Toolbox()

# add attribute generator to the toolbox
toolbox.register("indices", random.sample, range(TourSize), TourSize)

# initializers for individuals and population
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# genetic operators and fitness function
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
# toolbox.register("select", tools.selRoulette)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)


In [ ]:
random.seed(16)
pop = toolbox.population(n=500)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min)
algorithms.eaSimple(pop, toolbox, 0.7, 0.2, ngen=50, stats=stats)
# algorithms.eaSimple(pop, toolbox, 0.7, 0.2, ngen=50, stats=stats, verbose=False)

best = tools.selBest(pop, 1)[0]
print("Best: %s\nTotal Distance: %s" %(best.tolist(), best.fitness.values))


gen	nevals	min 
0  	500   	3369
1  	395   	3307
2  	384   	3261
3  	386   	3269
4  	402   	2938
5  	371   	2938
6  	372   	2697
7  	361   	2671
8  	388   	2671
9  	377   	2671
10 	391   	2487
11 	348   	2487
12 	379   	2487
13 	386   	2521
14 	357   	2521
15 	366   	2481
16 	358   	2481
17 	390   	2336
18 	376   	2457
19 	402   	2373
20 	366   	2373
21 	360   	2373
22 	375   	2278
23 	388   	2278
24 	385   	2199
25 	396   	2176
26 	367   	2176
27 	382   	2142
28 	368   	2175
29 	367   	2128
30 	366   	2115
31 	376   	2115
32 	389   	2088
33 	374   	2088
34 	381   	2088
35 	380   	2085
36 	377   	2085
37 	383   	2085
38 	380   	2085
39 	418   	2085
40 	380   	2085
41 	367   	2085
42 	352   	2085
43 	364   	2085
44 	388   	2085
45 	373   	2085
46 	393   	2085
47 	388   	2085
48 	377   	2085
49 	375   	2085
50 	379   	2085
Best: [1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3, 0, 15, 11, 8, 4]
Total Distance: (2085.0,)


## Discussions

The best solution found for the Traveling Salesman Problem (TSP) is represented by the sequence [1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3, 0, 15, 11, 8, 4]. This means that the salesman starts at city 1, then goes to city 9, followed by city 10, and so on, until reaching city 4. The total distance covered in this route is 2085. This solution is one of the most efficient routes discovered for this particular instance of the TSP.

Comparing  with the provided result OptTour = [0, 15, 11, 8, 4, 1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3], OptDistance = 2085. It is obvious that gained results is correct because have the same total distance. The routes are different this is because there can be multiple ways to arrange the cities while maintaining the same total distance.